In [ ]:
%matplotlib inline

#
# import utilitaires
#
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

import helpers as hlp
import helpers.dataset.PlantVillage as pv
import helpers.features as feat


#
# parametres dataset PlantVillage
#
config_overrides = hlp.MetaObject.from_json("config_overrides.json")
executor = hlp.create_thread_pool_executor()

pv_config = pv.Config(executor)
if not config_overrides is None:
    hlp.MetaObject.override_from_object(pv_config, 
                                        config_overrides.dataset)

print("pv_config")
display(vars(pv_config))


#
# obtenir le dataset PlantVillage
#
pv_dataset = pv.load(pv_config)
pv_dataframe = pv_dataset.dataframe

print("PlantVillage", pv_dataframe.shape)

In [ ]:
#
# get some images
#
if True:
    pv_image_indices = [random.randint(0, pv_dataframe.shape[0]) for _ in range(12)]
else:
    pv_image_indices = [46764, 21155, 18039, 7483, 6065, 39987, 841, 52365, 46308, 55725, 58270, 60308]

pv_image_labels = pv_dataframe.loc[pv_image_indices, "label"].values
pv_images = [pv_dataset.get_image(i) for i in pv_image_indices]

print(pv_image_indices)
print(pv_image_labels)

In [ ]:
if False:
    _, axes = plt.subplots(3, 4, figsize=(11, 3 * 3.5))

    for ax, image, label in zip(axes.flatten(), pv_images, pv_image_labels):
        label = label.replace("_", "\n")
        
        plt.sca(ax)
        plt.title(label)
        plt.imshow(image)

    plt.tight_layout()
    plt.show()

In [ ]:
_, axes = plt.subplots(12, 4, figsize=(11, 12 * 3.5))

for i, (image, label) in enumerate( zip(pv_images, pv_image_labels) ):
    label = label.replace("_", "\n")

    plt.sca(axes[i, 0])
    plt.title(label)
    plt.imshow(image)

    # view histogram
    if False:
        hr = cv2.calcHist([image], [0], None, [256], [0, 256])
        hg = cv2.calcHist([image], [1], None, [256], [0, 256])
        hb = cv2.calcHist([image], [2], None, [256], [0, 256])

        plt.sca(axes[i, 1])
        plt.plot(hr, color="r")
        plt.plot(hg, color="g")
        plt.plot(hb, color="b")

    # color quantization (background very different from plant)
    # not great, color bleeds and does not really separate background
    if False:
        # Define criteria = ( type, max_iter = 10 , epsilon = 1.0 )
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

        # Apply KMeans
        iii = np.float32( image.reshape( (-1, 3) ) )
        compactness, labels, centers = cv2.kmeans(iii, 3, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        centers = np.uint8(centers)
        res = centers[ labels.flatten() ]
        res2 = res.reshape((image.shape))

        plt.sca(axes[i, 2])
        plt.imshow(res2)

    # HSV: S could work but shadows break, H very very bad quality
    if False:
        hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        plt.sca(axes[i, 1])
        plt.imshow(hsv_img[:, :, 0], cmap='gray')

        plt.sca(axes[i, 2])
        plt.imshow(hsv_img[:, :, 1], cmap='gray')

        plt.sca(axes[i, 3])
        plt.imshow(hsv_img[:, :, 2], cmap='gray')

    # RGB
    if True:
        plt.sca(axes[i, 1])
        plt.imshow(image[:, :, 0], cmap='gray')

        plt.sca(axes[i, 2])
        plt.imshow(image[:, :, 1], cmap='gray')

        plt.sca(axes[i, 3])
        plt.imshow(image[:, :, 2], cmap='gray')

    if False:
        ii = np.float32(image) / 255.0

        blur0 = cv2.blur(ii, (5, 5))
        blur1 = cv2.blur(ii, (10, 10))

        plt.sca(axes[i, 1])
        plt.imshow(blur0)

        plt.sca(axes[i, 2])
        plt.imshow(np.abs(ii - blur0) * 10)

        plt.sca(axes[i, 3])
        plt.imshow(np.abs(blur1 - blur0) * 10)

plt.tight_layout()
plt.show()